In [140]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

from numpy import array


"""from keras.preprocessing.text import Tokenizer

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding"""


'from keras.preprocessing.text import Tokenizer\n\nfrom keras.preprocessing.text import one_hot\nfrom keras.preprocessing.sequence import pad_sequences\nfrom keras.models import Sequential\nfrom keras.layers.core import Activation, Dropout, Dense\nfrom keras.layers import Flatten\nfrom keras.layers import GlobalMaxPooling1D\nfrom keras.layers.embeddings import Embedding'

In [141]:
reviews = pd.read_csv("~/Desktop/ece684/movies.csv")
reviews


,text,label
0,I always wrote this series off as being a comp...,0
1,1st watched 12/7/2002 - 3 out of 10(Dir-Steve ...,0
2,This movie was so poorly written and directed ...,0
3,The most interesting thing about Miryang (Secr...,1
4,"when i first read about ""berlin am meer"" i did...",0
...,...,...
4995,This is the kind of picture John Lassiter woul...,1
4996,A MUST SEE! I saw WHIPPED at a press screening...,1
4997,NBC should be ashamed. I wouldn't allow my chi...,0
4998,This movie is a clumsy mishmash of various gho...,0


In [142]:
reviews.isnull().values.any()
# The data set has no missing values



False

In [143]:
reviews["text"][6]


"I saw a screening of this movie last night. I had high expectations going into it, but was definitely disappointed. Within 5 minutes of the opening, Williams is already campaigning for his presidency. And he becomes president in the first 40 minutes. So there goes all that aspect of the movie. The first half hour are hilarious. Don't get me wrong, the movie has its moments. But after the first half hour, it takes a turn for the worst. It becomes less of a comedy, and more of a thriller/drama/love story...which is pointless. the movie goes nowhere and stands still for a good 30 minutes. there are laughs interspersed here and there, but the consistently funny part is in the beginning and only the beginning. at one point, the biggest cheer i heard in the audience is when a person in the crowd yelled 'boooo' during a very confusingly emotional scene. Williams gives a great performance, right on par with his comedic style. Walken also delivers a strong supporting role as only he can. I thi

In [144]:
import re

# Reformats Reviews


def cleanReview(rev):
    # Removing anything within a HTML tag
    edited_rev = re.compile(r"<[^>]+>").sub(" ", rev)
    # Removing Punctuation
    edited_rev = re.sub(r"[^\w\s]", " ", edited_rev)
    # Removing Numbers
    edited_rev = re.sub(r"[0-9]", " ", edited_rev)
    # Removing single characters
    edited_rev = re.sub(r"\s+[a-zA-Z]\s+", " ", edited_rev)
    # Removing multiple spaces
    edited_rev = re.sub(r"\s+", " ", edited_rev)

    return edited_rev


In [145]:
# Checking Cleaning Function
"""rev1 = reviews["text"][3]
rev= cleanReview(rev1)
rev"""



'rev1 = reviews["text"][3]\nrev= cleanReview(rev1)\nrev'

In [146]:
review_list = []

# Adding all of the movie reviews to a list
for r in reviews["text"]:
    review_list.append(cleanReview(r))


In [147]:
#Start of tokenizing
words = " ".join(review_list)
words = list(words.split(" "))

In [148]:
#Getting word counts
totalCounts = {}
for j in range(len(words)):
    if words[j] not in totalCounts:
        totalCounts[words[j]] = 1
    if words[j] in totalCounts:
        totalCounts[words[j]] += 1

In [149]:
#Sorting dictionary to have highest word counts first
sorted_counts=dict(sorted(totalCounts.items(), key=lambda x: x[1], reverse=True))

In [150]:
#Making the highest word count value be 1, etc.
w_to_i = {w:i+1 for i, (w) in enumerate(sorted_counts)}

In [151]:
# Encoding the words by replacing the word with its given integer
X = []
for m in review_list:
    r = [w_to_i[w] for w in m.split()]
    X.append(r)

In [152]:
import statistics
lengths = []
for i in range(len(X)):
    l = len(X[i])
    lengths.append(l)
statistics.mean(lengths)

221.0492

In [160]:
#Padding for different review lengths
X_array = np.zeros((len(X), 220), dtype =int)
for i, rev2 in enumerate(X):
    rev_length = len(rev2)

    if rev_length <= 220:
        zeros = list(np.zeros(220-rev_length))
        mat = zeros + rev2
    elif rev_length > 220:
        mat = rev2[0:220]
    X_array[i,:] = np.array(mat)

In [ ]:
# Creating a training and test data sets
y = reviews["label"]

y = np.array(list(map(lambda x: 1 if x == 1 else 0, y)))

In [161]:
X_train, X_test, y_train, y_test = train_test_split(
    X_array, y, test_size=0.20, random_state=42
)